In [1]:
import torch
from transformers import (
    T5Tokenizer,
    MT5ForConditionalGeneration
)
from datasets import Dataset
import transformers
transformers.utils.logging.set_verbosity_error()

e:\EA-MTri\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
MODEL = 'E:/EA-MTri/results_mt5small_mt/checkpoint-74520'
BATCH_SIZE = 1
MAX_LENGTH = 256

In [6]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: ", DEVICE)

Device:  cuda


In [7]:
model = MT5ForConditionalGeneration.from_pretrained(MODEL)
tokenizer = T5Tokenizer.from_pretrained(MODEL, model_max_length=MAX_LENGTH)

In [8]:
model.to(DEVICE)

MT5ForConditionalGeneration(
  (shared): Embedding(250106, 512)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250106, 512)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
          

In [9]:
src = "en"

In [54]:
input_string = "entity translate English to German: Who can attend <ORG> Pontifical Ecclesiastical Academy </ORG> ?"
file_data = {src: [input_string]} 
test_data = Dataset.from_dict(file_data)

In [52]:
input_string = "recognize English named entities: Who can attend Pontifical Ecclesiastical Academy?"
file_data = {src: [input_string]} 
test_data = Dataset.from_dict(file_data)

In [56]:
input_string = "translate English to German: Who can attend Pontifical Ecclesiastical Academy?"
file_data = {src: [input_string]} 
test_data = Dataset.from_dict(file_data)

In [30]:
def generate_translation(batch):
    outputs = model.generate(
        **tokenizer(batch[src], return_tensors="pt", max_length=MAX_LENGTH, padding='max_length', truncation=True).to(DEVICE),
        max_length=MAX_LENGTH
    )
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch

In [55]:
results = test_data.map(generate_translation, batched=True, batch_size=BATCH_SIZE, remove_columns=[src])

pred_str = results["pred"]

for item in pred_str:
    print("".join(item))

Map: 100%|██████████| 1/1 [00:00<00:00,  3.34 examples/s]

Wer hat die Pontifical Ecclesiastika-Akademie besucht?


In [53]:
results = test_data.map(generate_translation, batched=True, batch_size=BATCH_SIZE, remove_columns=[src])

pred_str = results["pred"]

for item in pred_str:
    print("".join(item))

Map: 100%|██████████| 1/1 [00:00<00:00,  7.37 examples/s]

Who can attend <ORG> Pontifical Ecclesiastical Academy </ORG> ?


In [57]:
results = test_data.map(generate_translation, batched=True, batch_size=BATCH_SIZE, remove_columns=[src])

pred_str = results["pred"]

for item in pred_str:
    print("".join(item))

Map: 100%|██████████| 1/1 [00:00<00:00,  2.82 examples/s]

Wer hat die Pontifical Ecclesiastika Academy besucht?
